# Neural Networks: Drone Classification
##### Eric Leu, Josiah Brett, Jackson Krebsbach, Blake Harlow
##### May 2020

In this worksheet, we create an algorithm for training a neural network. We will then use it for classifying drone imagery and compare it to results obtained from package random forest algorithm.

In [17]:
# First, import the necessary packages.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping

## Data

First, we import the data that is in this same directory. These <code>.csv</code> files already contain the targets in the first column. The training data is seperate from the validation data, so we import the training and validation data as seperate dataframes. We then print the first five entries of the training dataframe in order to confirm that it was imported as expected.

In [2]:
# Create a data frame
TrainDataframe = pd.read_csv('./train.csv')
print(TrainDataframe.shape)
TestDataframe = pd.read_csv('./val.csv')
# Print the shapes
print(TestDataframe.shape)

(500, 7)
(500, 7)


In [3]:
TrainDataframe[:5]


,Class,Blue,Green,Red,RedEdge,NIR,Thermal
0,0.0,0.031719,0.076708,0.024127,0.005650,0.000000,0.958818
1,0.0,0.058563,0.102297,0.041034,0.017704,0.001276,0.957622
2,0.0,0.050540,0.089930,0.040690,0.013121,0.000000,0.958204
3,0.0,0.044991,0.082407,0.028196,0.008350,0.000000,0.957364
4,0.0,0.059913,0.103095,0.053527,0.025488,0.003190,0.957978


Notice that the first column contains the targets. Thus, we next assign the first column to be the targets and the remaining columns to be the parameters of the data.

In [4]:
TrainData = np.array(TrainDataframe.loc[:,'Blue':])
TestData = np.array(TestDataframe.loc[:,'Blue':])
print(TrainData.shape,TestData.shape)

(500, 6) (500, 6)


In [5]:
TrainVal = np.array(TrainDataframe.loc[:,'Class'])
TestVal = np.array(TestDataframe.loc[:,'Class'])
print(TrainVal.shape,TestVal.shape)

(500,) (500,)


At this point, we have the data in four different numpy arrays. The training data is in <code>TrainData</code> and has targets in <code>TrainVal</code> and the testing data is in <code>TestData</code> and has targets in <code>TestVal</code>.

## Algorithm

In order to begin coding the algorithm for the neural network, we first define some initial functions that will be used. First, we define the activation functions that we will use, the relu function and the softmax function. The former will be used as the activation function of the hidden layer and the later will be used as an activation function for the output layer.

In [6]:
#Activation function relu for forward propagation. 
#This includes both relu (type = 0) and
#its unit step function derivative (type = 1).
def relu(x,type):
    #Perform regular relu
    if(type==0):
        #Loop over columns
        for j in range(x.shape[1]):
            #Loop over rows
            for i in range(x.shape[0]):
                if x[i,j] < 0:
                    x[i,j] = 0
    #Perform relu derivative
    elif(type==1):
        #Loop over columns
        for j in range(x.shape[1]):
            #Loop over rows
            for i in range(x.shape[0]):
                if x[i,j] < 0:
                    x[i,j] = 0
                else:
                    x[i,j] = 1
    return x

In [7]:
#Activation function softmax for forward propagation. 
#The regular softmax function overflows whenever any input data is large due to the .exp() function.
#I've modified it to shift the input vector to ensure calculations are stable.
def softmax(x):
    z = x-np.max(x,axis=1).reshape(x.shape[0],1)
    return np.exp(z) / np.exp(z).sum(axis=1, keepdims=True)

Now, we code the algorithm that trains the neural network weights and biases. This function takes the training data along with its targets, along with a variety of hyperparamters. <code>numer_hiddennodes</code> gives the number of nodes in the hidden layer, <code>number_outputnodes</code> gives the number of nodes at the ouput layer, <code>learningrate</code> contributes to the step size, and <code>epochs</code> gives the number of iterations that the model completes. Notice that <code>number_outputnodes</code> must be equal to the expected dimesion of the output.

In [8]:
#Neural network for our classification problem.
#It is fit through a set of training data then outputs the final optimized weights and bias for training and validation.
def NeuralNetwork(TrainData,TrainVal,number_hiddennodes,number_outputnodes,learningrate = 0.001,epochs=3900):
    np.random.seed(8675309)
    bands = TrainData.shape[1]

    #We want to use one hot encoded classes in order to adjust the weights of an artificial neural network by applying error.
    OH_Encoded = np.zeros(shape=(TrainVal.shape[0],number_outputnodes)).astype(int)
    for index in range(TrainVal.shape[0]):
        OH_Encoded[index] = np.identity(number_outputnodes)[TrainVal[index].astype(int)].astype(int)

    #Initialize layer weights and bias. Keep in mind we want to take random values upon a normalized Gaussian distribution for bias to aid stochastic gradient descent.
    #Hidden layer: Relu
    relu_weights, relu_bias = np.random.rand(bands,number_hiddennodes),np.random.randn(number_hiddennodes)

    #Output layer: Softmax
    softmax_weights,softmax_bias = np.random.rand(number_hiddennodes,number_outputnodes),np.random.randn(number_outputnodes)

    #Remember that epochs is just the naming sense of how many times we train a model for artificial neural networks. You can just think of them as the same thing.

    for iteration in range(epochs):
        #Perform forward propagation
        Weighted_bands = np.dot(TrainData,relu_weights)+relu_bias
        Relu_layer = relu(Weighted_bands,0)
        Softmax_layer = softmax(np.dot(Relu_layer,softmax_weights)+softmax_bias)

        #Perform backpropagation
        softmax_weight_error = np.dot(Relu_layer.transpose(),Softmax_layer-OH_Encoded)
        softmax_bias_error = Softmax_layer-OH_Encoded
        Relu_layer_derivative = relu(Weighted_bands,1)            #Remember the derivative is the slope in stochastic gradient descent.
        relu_weight_error = np.dot(TrainData.transpose(),Relu_layer_derivative*np.dot(Softmax_layer-OH_Encoded,softmax_weights.transpose()))
        relu_bias_error = np.dot(Softmax_layer-OH_Encoded,softmax_weights.transpose())*Relu_layer_derivative

        #Adjust weights with error in relation to Training data. 0.001 is the standard learning rate for the layers of an artificial neural network.
        relu_weights -= learningrate*relu_weight_error
        softmax_weights -= learningrate*softmax_weight_error
        relu_bias -= learningrate*relu_bias_error.sum(axis=0)
        softmax_bias -= learningrate*softmax_bias_error.sum(axis=0)
    return relu_weights,relu_bias,softmax_weights,softmax_bias

#Accuracy. This takes the weights obtained through a Neural Network and uses them to classify a set of testing data.
def accuracy(weights, TestData, TestVal, TrainData, TrainVal):
    # Perform forward propagation For training set
    Relu_layer_train = relu(np.dot(TrainData, weights[0]) + weights[1], 0)
    Softmax_layer_train = softmax(np.dot(Relu_layer_train, weights[2]) + weights[3])
    predictionTrain = np.argmax(Softmax_layer_train, axis=1)

    # Perform forward propagation For test set
    Relu_layer_test = relu(np.dot(TestData, weights[0]) + weights[1], 0)
    Softmax_layer_test = softmax(np.dot(Relu_layer_test, weights[2]) + weights[3])
    predictionTest = np.argmax(Softmax_layer_test, axis=1)
    print('Training Accuracy: {:.3f}'.format(np.mean(predictionTrain == TrainVal)))
    print('Validation Accuracy: {:.3f}'.format(np.mean(predictionTest == TestVal)))
    return np.mean(predictionTest == TestVal)


#Classification function. This takes the weights obtained through a Neural Network and uses them to classify a set of testing data.
def classify(weights, TestData):
    
    #Perform forward propagation
    Relu_layer = relu(np.dot(TestData,weights[0])+weights[1],0)
    Softmax_layer = softmax(np.dot(Relu_layer,weights[2])+weights[3])
    prediction = np.argmax(Softmax_layer,axis=1)

    return prediction

Now we can finally train a neural network on our orthomosaic data.

In [9]:
TrainedNetwork = NeuralNetwork(TrainData,TrainVal,number_hiddennodes=6,number_outputnodes=5)
prediction = accuracy(TrainedNetwork,TestData,TestVal, TrainData, TrainVal)

Training Accuracy: 0.996
Validation Accuracy: 0.982


##### If you want to try to find a better accuracy, the important hyperparameters are <b> number_hiddennodes, learningrate, epochs.</b>

In [10]:
#TrainedNetwork = NeuralNetwork(TrainData,TrainVal,number_hiddennodes=6,number_outputnodes=3, epochs = 10000)
#prediction = classify(TrainedNetwork,TestData,TestVal, TrainData, TrainVal)

In [11]:
forest = RandomForestClassifier(random_state=0)                    #Create and fit model
forest.fit(TrainData, TrainVal)
print("Accuracy on training set: {:.3f}\nAccuracy on test set: {:.4f}".format(forest.score(TrainData, TrainVal),forest.score(TestData, TestVal)))        #Display scores of classifier

Accuracy on training set: 1.000
Accuracy on test set: 0.9860


#### This is the best random forest model's scores. Keep in mind that we're only using a one hidden layer ANN.

In [12]:
#To work with the PCA data change the file path to where the 
#TIFF and shape file is stored locally on your machine
ortho_file = './ortho.tif'
shp_file = './cropped.shp'

## Classification

We will take a cropped region and use our trained neural network to classify every pixel using forward propogation. The following code reads in a shape file which encompasses the desired region and then crops the orthomosaic. The pixels are then extracted and then the classify function is used to classify the pixels. The classified image is reshaped and written as a geotiff to be viewed.

In [18]:
shp = gpd.read_file(shp_file)
geoms = shp.geometry.values
geoms = [mapping(geoms[0])]

with rasterio.open(ortho_file) as src:
    out_image, out_transform = mask(src,geoms,crop =True)
    
data = out_image[0]
row, col = np.where(data != 0 )
bands = out_image.shape[0]

RasterioIOError: ./ortho.tif: No such file or directory

In [ ]:
Y = np.zeros((row.size,bands))
for ba in range(bands):
    Y[:, ba] = np.extract(data != 0, out_image[ba])

In [ ]:
for x in range(Y.shape[1]):        
    maxY = np.max(Y[:,x])
    Y[:,x] = Y[:,x]/maxY

In [ ]:
classifiedImage = classify(TrainedNetwork,Y)
C_image = np.copy(out_image)[0]
C_image[row,col] = classifiedImage

In [ ]:
# Create a new data set and write it
new_dataset = rasterio.open(
    './classified.tif',
    'w',
    driver = 'GTiff',
    height = C_image.shape[0],
    width = C_image.shape[1],
    count = 1,
    crs = src.crs,
    dtype = src.dtypes[0],
    transform = out_transform,
    nodata = 8,
    )
# write the new file
new_dataset.write(C_image,1)